<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/examples/citation/pdf_page_reference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [1]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [2]:
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    download_loader,
    RAKEKeywordTableIndex,
)

In [6]:
import os
os.environ["OPENAI_API_KEY"] = ""

Set service context to enable streaming

In [7]:
from llama_index import LLMPredictor, ServiceContext
from llama_index.llms import OpenAI

service_context = ServiceContext.from_defaults(
    llm=OpenAI(temperature=0, model="text-davinci-003")
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Download Data

In [8]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2023-11-05 20:26:00--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1440303 (1.4M) [application/octet-stream]
Saving to: ‘data/10k/lyft_2021.pdf’

data/10k/lyft_2021. 100%[===================>]   1.37M  --.-KB/s    in 0.07s   

2023-11-05 20:26:00 (20.8 MB/s) - ‘data/10k/lyft_2021.pdf’ saved [1440303/1440303]



Load document and build index

In [9]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 5.4 MB/s eta 0:00:00


In [10]:
reader = SimpleDirectoryReader(input_files=["./data/10k/lyft_2021.pdf"])
data = reader.load_data()

In [11]:
print(data[1])
print(os.getcwd())

Doc ID: 98e103cf-0668-4254-80cb-1ba7c96504db
Text: Table of ContentsPage PART I Item 1. Business 5 Item 1A. Risk
Factors 15 Item 1B. Unresolved Staff Comments 53 Item 2. Properties 53
Item 3. Legal Proceedings 54 Item 4. Mine Safety Disclosures 54 PART
II Item 5. Market for Registrant’s Com mon Equity, Related Stockholder
Matters and Issuer Purchases of Equity Securities55 Item 6. [Reserved]
56 ...
/content


In [12]:
ls

data/  sample_data/


In [15]:
import time
#from gensim.similarities import VectorStoreIndex

#data = [...]  # Your list of documents
service_context = {}  # Your service context, if applicable
batch_size = 100  # Adjust the batch size as needed
delay_seconds = 5  # Adjust the delay time as needed

index = VectorStoreIndex.from_documents(data, service_context=service_context)
for i in range(0, len(data), batch_size):
    batch = data[i:i + batch_size]
    # Index the batch
    batch_index = VectorStoreIndex.from_documents(batch, service_context=service_context)
    index.add_documents_from(batch_index)
    time.sleep(delay_seconds)
    # You can perform operations on the index here


RateLimitError: ignored

In [13]:
index = VectorStoreIndex.from_documents(data, service_context=service_context)

RateLimitError: ignored

In [ ]:
query_engine = index.as_query_engine(streaming=True, similarity_top_k=3)

Stream response with page citation

In [ ]:
response = query_engine.query(
    "What was the impact of COVID? Show statements in bullet form and show"
    " page reference after each statement."
)
response.print_response_stream()

Inspect source nodes

In [ ]:
for node in response.source_nodes:
    print("-----")
    text_fmt = node.node.get_content().strip().replace("\n", " ")[:1000]
    print(f"Text:\t {text_fmt} ...")
    print(f"Metadata:\t {node.node.metadata}")
    print(f"Score:\t {node.score:.3f}")